<a href="https://colab.research.google.com/github/SeounghwanPark/nesaram/blob/main/%EC%9D%BC%EC%A0%95%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("project_schedule1.csv")

# 데이터 미리보기
print(df.head())

In [ ]:
# 데이터 타입 확인
print(df.info())

# 결측치 확인
print(df.isnull().sum())

In [ ]:
# 분석에 필요 없는 컬럼 삭제
df = df[['Activity_ID', 'Start_Date', 'Finish_Date', 'Planned_Finish', 'Actual_Finish', 'Status']]

In [ ]:
# 날짜 컬럼을 datetime 형식으로 변환
df['Start_Date'] = pd.to_datetime(df['Start_Date'])
df['Finish_Date'] = pd.to_datetime(df['Finish_Date'])
df['Planned_Finish'] = pd.to_datetime(df['Planned_Finish'])
df['Actual_Finish'] = pd.to_datetime(df['Actual_Finish'])

In [ ]:
# 일정 지연 일수 계산
df['Delay_Days'] = (df['Actual_Finish'] - df['Planned_Finish']).dt.days
df['Delay_Flag'] = df['Delay_Days'].apply(lambda x: 1 if x > 0 else 0)  # 지연 여부 (1: 지연, 0: 정상)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(df['Delay_Days'], bins=30, kde=True)
plt.title("일정 지연 분포")
plt.xlabel("지연 일수")
plt.ylabel("빈도수")
plt.show()

In [ ]:
df['Month'] = df['Planned_Finish'].dt.to_period('M')

# 월별 평균 지연 일수 계산
monthly_delay = df.groupby('Month')['Delay_Days'].mean()

plt.figure(figsize=(12, 5))
monthly_delay.plot(kind='bar')
plt.title("월별 평균 일정 지연")
plt.xlabel("월")
plt.ylabel("평균 지연 일수")
plt.show()

In [ ]:
# 1: 머신러닝을 위한 데이터 준비
from sklearn.model_selection import train_test_split

# Feature & Target 설정
X = df[['Start_Date', 'Planned_Finish']]  # 독립변수
y = df['Delay_Flag']  # 목표 변수 (지연 여부 예측)

# 날짜를 숫자로 변환
X['Start_Date'] = X['Start_Date'].astype('int64') // 10**9
X['Planned_Finish'] = X['Planned_Finish'].astype('int64') // 10**9

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 2: 랜덤 포레스트 모델 학습
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 정확도 평가
print("정확도:", accuracy_score(y_test, y_pred))

In [ ]:
# 시계열 분석을 활용한 일정 지연 예측 (ARIMA, Prophet)
# 1: ARIMA 모델 학습
from statsmodels.tsa.arima.model import ARIMA

# 시계열 데이터 준비
ts_data = df.groupby('Planned_Finish')['Delay_Days'].mean()

# ARIMA 모델 학습
model = ARIMA(ts_data, order=(1,1,1))
model_fit = model.fit()

# 예측
forecast = model_fit.forecast(steps=10)
print(forecast)

In [ ]:
# 2: Prophet 모델 학습
from prophet import Prophet

# Prophet 데이터 준비
prophet_df = df[['Planned_Finish', 'Delay_Days']].rename(columns={'Planned_Finish': 'ds', 'Delay_Days': 'y'})

# 모델 학습
model = Prophet()
model.fit(prophet_df)

# 예측
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# 결과 시각화
model.plot(forecast)
plt.show()